# This program connects to Box.com, gets all of the folders and images and stores information into the database.

https://developer.box.com/en/guides/authentication/oauth2/with-sdk/

In [1]:
!pip install boxsdk
!pip install auth
!pip install redis
!pip install mysql.connector
!pip install requests

# Run the cell below to authenticate with Box.com so we can use the API

In [1]:
from boxsdk import OAuth2

import json
#Set the file we want to use for authenticating a Box app
#The json file stores the client_id and client_secret so we don't have it in the code.
# The json file looks like this:
#{
#"client_id":"___the_codes_for_client_id___",
#"client_secret":"___the_codes_for_client_secret___"
#}

oauth_settings_file = 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\box_app_test.json'
with open(oauth_settings_file, "r") as read_file:
    oauth_data = json.load(read_file)
print(oauth_data["client_id"])
print(oauth_data["client_secret"])

oauth = OAuth2(
    client_id=oauth_data["client_id"],
    client_secret=oauth_data["client_secret"]
)

auth_url, csrf_token = oauth.get_authorization_url('https://jeffblackadar.ca')
print("Click on this:")
print(auth_url)
print(csrf_token)
print("Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.")


rvy5o5ic72c00ldi8vpcqmof0t7h6w85
VWJQ1LTracitLw5HfG8Q0TKplBogFEIT
Click on this:
https://account.box.com/api/oauth2/authorize?state=box_csrf_token_L0jHQQyqF1Uvwyeb&response_type=code&client_id=rvy5o5ic72c00ldi8vpcqmof0t7h6w85&redirect_uri=https%3A%2F%2Fjeffblackadar.ca
box_csrf_token_L0jHQQyqF1Uvwyeb
Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.


In [2]:
from boxsdk import Client

# Make sure that the csrf token you get from the `state` parameter
# in the final redirect URI is the same token you get from the
# get_authorization_url method to protect against CSRF vulnerabilities.
#assert 'THE_CSRF_TOKEN_YOU_GOT' == csrf_token
access_token, refresh_token = oauth.authenticate('KAySe0WadihUhicwol6OsrIsj5E2L484')
client = Client(oauth)

## Test the connection to Box.com - list all folders and files using recursion

In [ ]:
def process_subfolder_test(client, folder_id, folder_name):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder_test(client, item.id,folder_name+"/"+item.name)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))            
    return

process_subfolder_test(client, '0',"")

## Get all folders and store all files into a database.

In [17]:
def process_subfolder(client, folder_id, folder_name,cursor_insert):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder(client, item.id,folder_name+"/"+item.name,cursor_insert)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))
            
            insert_image = ("INSERT INTO tbl_box_images (`id_box_file`,`id_box_folder`,`folder`,`file_name`,`img_src`,`img_url`) VALUES (%s,%s,%s,%s,%s,%s);")
            
            cursor_insert.execute(insert_image, (item.id, folder_id, folder_name, item.name, folder_name+"/"+item.name, ""))
            mariadb_connection.commit()
    return





In [ ]:
import requests
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'


import mysql.connector as mariadb

mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")

cursor_insert = mariadb_connection.cursor(buffered=True)

process_subfolder(client, '0',"", cursor_insert)
    
mariadb_connection.close()    


In [20]:
mariadb_connection.close()    

## Process images: Connect to Box.com, download image, get its hash value.

In [3]:
# Image hash functions
# import the necessary packages
from imutils import paths
import argparse
import time
import sys
import cv2
import os

# hashing with OpenCV and PythonPython
def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))

    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]

    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))
 
    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]
 
    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

In [6]:
download_process_box_thumbnail(client,3063, "580416299144", "/pinp5/Maps/Maps", "Plan Bay of Naples 1865 from Karl von Spruner’s 1865 rendering of Southern Italy and Sicily in antiquity Wikimedia.jpg", cursor_update)

NameError: name 'cursor_update' is not defined

In [12]:
import os
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'

# using the thumbnail for image hashing does not work   
def download_process_box_full_id_box_file(client, id_box, id_box_file, folder, file_name, cursor_update):
    #Some file names can't be processed.  This uses the box file name instead.
    print("this file: "+id_box_file+" "+file_name)
    #using the the large 320 pixel thumbnail produces different results than the full image - it does not work
    #try:
    #file_info = client.file(id_box_file).get()
    file_name = id_box_file+".jpg"
    imagecontent = client.file(id_box_file).content()
    # load the image for the image hash later.

    img_local_path=base_dir+folder+"/"+file_name
    #print(img_local_path)
    img_file_name = file_name
    img_local_folder = base_dir+folder

    #see if the image is there already
    if not os.path.exists(img_local_path):
        img_file_name = file_name
        img_local_folder = base_dir+folder
        
        if not os.path.exists(img_local_folder):
            #os.mkdir(img_local_folder)
            path = Path(img_local_folder)
            path.mkdir(parents=True,exist_ok=True)
            #print(img_local_folder+'/'+ img_file_name)
            
            # Open the url image, set stream to True, this will return the stream content.
            # Open a local file with wb ( write binary ) permission.
            #local_file = open(img_local_folder+'/'+ img_file_name, 'wb')
        with open(img_local_folder+'/'+ img_file_name,'wb') as f:
                f.write(imagecontent)
            #print(thumbnail)
        del imagecontent
        f.close()
            #image hash
    imageHash = 0
    #print("image hash")
    if not os.path.exists(img_local_path):
        print("Error image not loaded for image hash   " + img_local_path)
    else:         
        # load the image from disk
        path = Path(img_local_folder)
        os.chdir(path)
        hash_image = cv2.imread(img_file_name)
 
        # if the image is None then we could not load it from disk (so skip it)
        if not hash_image is None:
            # convert the image to grayscale and compute the hash
            hash_image = cv2.cvtColor(hash_image, cv2.COLOR_BGR2GRAY)
            imageHash = dhash(hash_image)
            #print(imageHash)
        else:
            imageHash = 0
            #print("image hash - can't find image?")
        update_image = ("UPDATE tbl_box_images SET image_hash = %s WHERE id = %s;")
        print(id_box, imageHash)
        cursor_update.execute(update_image, (imageHash, id_box))
        mariadb_connection.commit()
        os.remove(img_local_path)
    return
    #except:
    #    print(sys.exc_info()[0])
    #    return
    

In [ ]:
number_of_rows_to_run = 100000
start_id = 59607
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_inner = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src FROM webpage_images.tbl_box_images WHERE id>=%s AND id<%s", (start_id, end_id))
for id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src in cursor:
    #print(id, id_box_file, img_path_match, id_tbl_webpage_images, img_src)
    #don't include .xml, .htm* or .thmx
    #for row 158348
    if(img_path_match is None):
        img_path_match = ""
    if (img_src.lower().find(".xml")<0) and (img_src.lower().find(".htm")<0) and (img_src.lower().find(".thmx")<0) and (img_src.lower().find(".js")<0) and (img_src.lower().find(".pdf")<0):
        download_process_box_full_id_box_file(client, id, str(id_box_file), folder, file_name, cursor_update)
        if(id_tbl_webpage_images is not None):
            if(not id_tbl_webpage_images>0):
                #logic repeats below - making a function seemed to make this more complicated
                #No match found in the table tbl_box_images
                wi_id=0            
                cursor_wi_id = mariadb_connection.cursor(buffered=True, dictionary=True)
                img_path_match_sql = img_path_match.replace("'","''")
                #print(img_path_match_sql)
                cursor_wi_id.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match='"+img_path_match_sql+"';")
                #print(cursor_inner)
                #print("rows "+str(cursor_wi_id.rowcount))
                for wi_ids in cursor_wi_id:
                    wi_id=wi_ids["wi_id"]
                    print("id "+str(wi_id))
                    if(wi_id>0):
                        update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                        #print(update_image)
                        cursor_update.execute(update_image, (wi_id, id))
                        mariadb_connection.commit()
                cursor_wi_id.close()
        else:
            #logic repeats from above
            wi_id=0            
            cursor_wi_id = mariadb_connection.cursor(buffered=True, dictionary=True)
            #cursor_inner.execute(select_match, (img_path_match))
            
            img_path_match_sql = img_path_match.replace("'","''")
            #print(img_path_match_sql)
            cursor_wi_id.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match='"+img_path_match_sql+"';")
            #print("rows "+str(cursor_wi_id.rowcount))
            for wi_ids in cursor_wi_id:
                wi_id=wi_ids["wi_id"]
                print("id "+str(wi_id))
                if(wi_id>0):
                    update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                    #print(update_image)
                    cursor_update.execute(update_image, (wi_id, id))
                    mariadb_connection.commit()
            cursor_wi_id.close()
            
cursor.close()
cursor_update.close()
cursor_inner.close()
mariadb_connection.close()    
#/Altars/Altar 10110_files/image009.jpg
#/Altars/Altar 10110_files/image009.jpg

In [47]:
mariadb_connection.close()    

In [ ]:
#Clean up the 0
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_inner = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src FROM webpage_images.tbl_box_images WHERE image_hash is null= 0")
for id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src in cursor:
    print(id, id_box_file, file_name, img_path_match, id_tbl_webpage_images, img_src)
    #don't include .xml, .htm* or .thmx
    #for row 158348
    if(img_path_match is None):
        img_path_match = ""
    if (img_src.lower().find(".xml")<0) and (img_src.lower().find(".htm")<0) and (img_src.lower().find(".thmx")<0) and (img_src.lower().find(".js")<0) and (img_src.lower().find(".pdf")<0):
        download_process_box_thumbnail_id_box_file(client, id, str(id_box_file), folder, file_name, cursor_update)
        if(id_tbl_webpage_images is not None):
            if(not id_tbl_webpage_images>0):
                #logic repeats below - making a function seemed to make this more complicated
                #No match found in the table tbl_box_images
                wi_id=0            
                cursor_wi_id = mariadb_connection.cursor(buffered=True, dictionary=True)
                img_path_match_sql = img_path_match.replace("'","''")
                #print(img_path_match_sql)
                cursor_wi_id.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match='"+img_path_match_sql+"';")
                #print(cursor_inner)
                #print("rows "+str(cursor_wi_id.rowcount))
                for wi_ids in cursor_wi_id:
                    wi_id=wi_ids["wi_id"]
                    print("id "+str(wi_id))
                    if(wi_id>0):
                        update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                        #print(update_image)
                        cursor_update.execute(update_image, (wi_id, id))
                        mariadb_connection.commit()
                cursor_wi_id.close()
                #if(wi_id<1):
                #    download_process_box_thumbnail(client, id, str(id_box_file), folder, file_name, cursor_update)
        else:
            #logic repeats from above
            wi_id=0            
            cursor_wi_id = mariadb_connection.cursor(buffered=True, dictionary=True)
            #cursor_inner.execute(select_match, (img_path_match))
            
            img_path_match_sql = img_path_match.replace("'","''")
            #print(img_path_match_sql)
            cursor_wi_id.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match='"+img_path_match_sql+"';")
            #print("rows "+str(cursor_wi_id.rowcount))
            for wi_ids in cursor_wi_id:
                wi_id=wi_ids["wi_id"]
                print("id "+str(wi_id))
                if(wi_id>0):
                    update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                    #print(update_image)
                    cursor_update.execute(update_image, (wi_id, id))
                    mariadb_connection.commit()
            cursor_wi_id.close()
            #if(wi_id<1):
            #    download_process_box_thumbnail(client, id, str(id_box_file), folder, file_name, cursor_update)
            
cursor.close()
cursor_update.close()
cursor_inner.close()
mariadb_connection.close()    



In [ ]:
mariadb_connection.close()   